## Find difference in the processing of derivative contracts using GPT-4

#### Imports 

In [1]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import logging
import yaml
import os

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup

In [3]:
with open('./credentials/oai-key.yml', 'rb') as f:
    credentials = yaml.safe_load(f)
    
api_key = credentials['key']
os.environ['OPENAI_API_KEY'] = api_key

In [4]:
LOCAL_INPUT_DIR = './DATA/INPUT'
LOCAL_OUTPUT_DIR = './DATA/OUTPUT'

MODEL_NAME = 'gpt-4'

FILE_NAME_1 = 'file-1'
FILE_NAME_2 = 'file-2'

#### Find differences 

In [5]:
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME_1}/SUMMARY/{FILE_NAME_1}-summary-vai.txt', 'rb') as f:
    summary_1 = f.read()

with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME_2}/SUMMARY/{FILE_NAME_2}-summary-vai.txt', 'rb') as f:
    summary_2 = f.read()

In [6]:
llm = ChatOpenAI(model_name=MODEL_NAME, 
                 temperature=0.0, 
                 max_tokens=2048)

In [7]:
system_template = """You are a Derivatives Risk Analyst tasked with comparing the summaries of two documents, SUMMARY_1 and SUMMARY_2 provided below.
---
SUMMARY_1 => {summary_1}
---
SUMMARY_2 => {summary_2}
---
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = """First identify common themes between them. 
Next, point out the differences between the identified topics.
Finally, write a detailed paragraph on the identified differences."""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
prompt = chat_prompt.format_prompt(summary_1=summary_1, summary_2=summary_2).to_messages()

In [8]:
response = llm(prompt)

In [9]:
logger.info(response.content)

Common Themes:
1. Calculation of Adjusted Notional Amount: Both summaries discuss the calculation of the adjusted notional amount for different types of derivative contracts, including equity, commodity, interest rate, credit, and exchange rate derivative contracts.
2. Maturity Factor: Both summaries provide details on how to calculate the maturity factor of a derivative contract that is not subject to a variation margin agreement.
3. Treatment of Cleared Transactions: Both summaries discuss the treatment of cleared transactions that are not subject to a variation margin agreement.
4. Supervisory Duration: Both summaries discuss the calculation of the supervisory duration for an interest rate or credit derivative contract.

Differences:
1. Haircuts for Market Price Volatility: SUMMARY_1 provides detailed information on the haircuts for market price volatility based on different residual maturities and exposure types, while SUMMARY_2 does not mention this topic.
2. Minimum Period of Ris